<div class="alert alert-danger">
    <h4 style="font-weight: bold; font-size: 28px;">Feature Engineering</h4>
    <p style="font-size: 20px;">Basketball Reference Data (2000-2024)</p>
</div>

<a name="Feature Engineering"></a>

# Table of Contents

[Setup](#Setup)

[Data](#Data)

**[1. Create Team Matchups with Boxscores](#1.-Create-Team-Matchups-with-Boxscores)**

- [1.1. Merge Matchups with Team Box Scores](#1.1.-Merge-Matchups-with-Team-Box-Scores)

  - [1.1.1. Home Team](#1.1.1.-Home-Team)
 
  - [1.1.2. Away Team](#1.1.2.-Away-Team)
 
  - [1.1.3. Combine Home and Away Teams](#1.1.3.-Combine-Home-and-Away-Teams)
  
**[2. Create Team Targets](#2.-Create-Team-Targets)**

- [2.1. Game Winner](#2.1.-Game-Winner)

- [2.2. Total Points (over / under)](#2.2.-Total-Points-(over-/-under))

- [2.3. Difference in Points (plus / minus)](#2.3.-Difference-in-Points-(plus-/-minus))

**[3. Additional Team Level Features](#3.-Additional-Team-Level-Features)**
  
**[4. Player Level Features](#3.-Player-Level-Features)**

**[5. Time Windowed Features](#4.-Time-Windowed-Features)**

# Setup

[Return to top](#Feature-Engineering)

In [84]:
# basic modules
import os
import time
import random as rn
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# plotting style
plt.style.use('seaborn-v0_8-notebook')
sns.set_style('white')
#sns.set_style('darkgrid')

# pandas tricks for better display
pd.options.display.max_columns = 50  
pd.options.display.max_rows = 500     
pd.options.display.max_colwidth = 100
pd.options.display.precision = 3

# preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# warnings
import warnings
warnings.filterwarnings("ignore")

# user defined functions
import utility_functions as utl

# Data

[Return to top](#Feature-Engineering)

In [148]:
schedule_df = pd.read_csv('../data/original/br_season_schedule_2000_2024.csv')
team_bs_df = pd.read_csv('../data/original/br_team_box_scores_2000_2024.csv')

In [149]:
# convert the 'date' to datetime
schedule_df['date'] = pd.to_datetime(schedule_df['date'])
team_bs_df['date'] = pd.to_datetime(team_bs_df['date'])

In [133]:
schedule_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30752 entries, 0 to 30751
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   season           30752 non-null  int64         
 1   date             30752 non-null  datetime64[ns]
 2   start_time       30752 non-null  object        
 3   home_team        30752 non-null  object        
 4   away_team        30752 non-null  object        
 5   home_team_score  30752 non-null  int64         
 6   away_team_score  30752 non-null  int64         
 7   year_quarter     30752 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 1.9+ MB


In [134]:
team_bs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62466 entries, 0 to 62465
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   date                               62466 non-null  datetime64[ns]
 1   team                               62466 non-null  object        
 2   outcome                            62466 non-null  object        
 3   minutes_played                     62466 non-null  int64         
 4   made_field_goals                   62466 non-null  int64         
 5   attempted_field_goals              62466 non-null  int64         
 6   made_three_point_field_goals       62466 non-null  int64         
 7   attempted_three_point_field_goals  62466 non-null  int64         
 8   made_free_throws                   62466 non-null  int64         
 9   attempted_free_throws              62466 non-null  int64         
 10  offensive_rebounds                

<a name="1.-Create-Team-Matchups-with-Boxscores"></a>
# 1. Create Team Matchups with Boxscores

[Return to top](#Feature-Engineering)

<a name="1.1.-Merge-Matchups-with-Team-Box-Scores"></a>
## 1.1. Merge Matchups with Team Box Scores

[Return to top](#Feature-Engineering)

We need to merge the data frame with the matchups `schedule_df` and the data frame with the team features `team_bs_df`.

<a name="1.1.1.-Home-Team"></a>
### 1.1.1. Home Team

[Return to top](#Feature-Engineering)

In [150]:
# step 1: merge for home teams

# a) merge team_bs_df with schedule_df on 'team' and 'date' where team_bs_df's team is the home team in schedule_df
home_team_df = pd.merge(schedule_df, team_bs_df, how='left', 
                        left_on=['date', 'home_team', 'home_team_score'], 
                        right_on=['date', 'team', 'points'])

# b) drop redundant 'teams' column
home_team_df.drop(columns=['team', 'points'], inplace=True)

# c) rename columns to indicate they are for the home team

# get list of columns to prefix with 'home_'
home_prefix_cols = [col for col in home_team_df.columns if col not in schedule_df.columns]

# create a renaming mapping
home_rename_dict = {col: f'home_{col}' for col in home_prefix_cols}

# rename the columns with 'home_' prefix
home_team_df.rename(columns=home_rename_dict, inplace=True)

In [151]:
home_team_df.head()

,season,date,start_time,home_team,away_team,home_team_score,away_team_score,home_outcome,home_minutes_played,home_made_field_goals,home_attempted_field_goals,home_made_three_point_field_goals,home_attempted_three_point_field_goals,home_made_free_throws,home_attempted_free_throws,home_offensive_rebounds,home_defensive_rebounds,home_assists,home_steals,home_blocks,home_turnovers,home_personal_fouls
0,2000,1999-11-02,1999-11-02 05:00:00+00:00,Charlotte Hornets,Orlando Magic,100,86,Win,240.0,34.0,82.0,10.0,22.0,22.0,37.0,14.0,33.0,25.0,13.0,5.0,21.0,25.0
1,2000,1999-11-02,1999-11-02 05:00:00+00:00,Dallas Mavericks,Golden State Warriors,108,96,Win,240.0,39.0,85.0,2.0,15.0,28.0,34.0,11.0,30.0,22.0,11.0,6.0,13.0,19.0
2,2000,1999-11-02,1999-11-02 05:00:00+00:00,Denver Nuggets,Phoenix Suns,107,102,Win,265.0,40.0,89.0,8.0,18.0,19.0,26.0,13.0,42.0,25.0,6.0,16.0,18.0,17.0
3,2000,1999-11-02,1999-11-02 05:00:00+00:00,Houston Rockets,Milwaukee Bucks,93,98,Loss,240.0,34.0,75.0,6.0,20.0,19.0,29.0,15.0,27.0,22.0,5.0,7.0,18.0,24.0
4,2000,1999-11-02,1999-11-02 05:00:00+00:00,Los Angeles Clippers,Seattle Supersonics,92,104,Loss,240.0,35.0,78.0,6.0,16.0,16.0,27.0,12.0,33.0,14.0,4.0,9.0,17.0,24.0


<a name="1.1.2.-Away-Team"></a>
### 1.1.2. Away Team

[Return to top](#Feature-Engineering)

In [152]:
# step 2: merge for away teams

# a) merge team_bs_df with schedule_df on 'team' and 'date' where team_bs_df's team is the away team in schedule_df
away_team_df = pd.merge(schedule_df, team_bs_df, how='left', 
                        left_on=['date', 'away_team', 'away_team_score'], 
                        right_on=['date', 'team', 'points'])

# b) drop redundant 'teams' column
away_team_df.drop(columns=['team', 'points'], inplace=True)

# c) rename columns to indicate they are for the away team

# get list of columns to prefix with 'away_'
away_prefix_cols = [col for col in away_team_df.columns if col not in schedule_df.columns]

# create a renaming mapping
away_rename_dict = {col: f'away_{col}' for col in away_prefix_cols}

# rename the columns with 'away_' prefix
away_team_df.rename(columns=away_rename_dict, inplace=True)

In [153]:
away_team_df.head()

,season,date,start_time,home_team,away_team,home_team_score,away_team_score,away_outcome,away_minutes_played,away_made_field_goals,away_attempted_field_goals,away_made_three_point_field_goals,away_attempted_three_point_field_goals,away_made_free_throws,away_attempted_free_throws,away_offensive_rebounds,away_defensive_rebounds,away_assists,away_steals,away_blocks,away_turnovers,away_personal_fouls
0,2000,1999-11-02,1999-11-02 05:00:00+00:00,Charlotte Hornets,Orlando Magic,100,86,Loss,240.0,34.0,80.0,0.0,8.0,18.0,27.0,12.0,36.0,17.0,9.0,4.0,22.0,31.0
1,2000,1999-11-02,1999-11-02 05:00:00+00:00,Dallas Mavericks,Golden State Warriors,108,96,Loss,240.0,34.0,82.0,7.0,18.0,21.0,27.0,18.0,33.0,14.0,4.0,2.0,23.0,29.0
2,2000,1999-11-02,1999-11-02 05:00:00+00:00,Denver Nuggets,Phoenix Suns,107,102,Loss,265.0,43.0,111.0,9.0,22.0,7.0,14.0,21.0,37.0,29.0,9.0,5.0,13.0,28.0
3,2000,1999-11-02,1999-11-02 05:00:00+00:00,Houston Rockets,Milwaukee Bucks,93,98,Win,240.0,38.0,79.0,3.0,7.0,19.0,28.0,10.0,25.0,20.0,10.0,2.0,8.0,25.0
4,2000,1999-11-02,1999-11-02 05:00:00+00:00,Los Angeles Clippers,Seattle Supersonics,92,104,Win,240.0,37.0,93.0,7.0,20.0,23.0,32.0,19.0,34.0,25.0,8.0,3.0,11.0,21.0


<a name="1.1.3.-Combine-Home-and-Away-Teams"></a>
### 1.1.3. Combine Home and Away Teams

[Return to top](#Feature-Engineering)

In [154]:
# step 3: combine home and away data

# identify columns in 'away_df' that are not in 'schedule_df'
# columns_to_merge = away_team_df.columns.difference(['season', 'start_time'])

# columns to merge on
columns_to_merge_on = ['season', 'start_time', 'date', 'home_team', 'away_team', 'home_team_score', 'away_team_score']

# merge 'home_team_df' with these selected columns from 'away_team_df'
# team_matchups_df = pd.merge(home_team_df, away_team_df[columns_to_merge], how='left', on=['date', 'home_team', 'away_team'])
team_matchups_df = pd.merge(home_team_df, away_team_df, how='left', on=columns_to_merge_on)

In [155]:
team_matchups_df.head()

,season,date,start_time,home_team,away_team,home_team_score,away_team_score,home_outcome,home_minutes_played,home_made_field_goals,home_attempted_field_goals,home_made_three_point_field_goals,home_attempted_three_point_field_goals,home_made_free_throws,home_attempted_free_throws,home_offensive_rebounds,home_defensive_rebounds,home_assists,home_steals,home_blocks,home_turnovers,home_personal_fouls,away_outcome,away_minutes_played,away_made_field_goals,away_attempted_field_goals,away_made_three_point_field_goals,away_attempted_three_point_field_goals,away_made_free_throws,away_attempted_free_throws,away_offensive_rebounds,away_defensive_rebounds,away_assists,away_steals,away_blocks,away_turnovers,away_personal_fouls
0,2000,1999-11-02,1999-11-02 05:00:00+00:00,Charlotte Hornets,Orlando Magic,100,86,Win,240.0,34.0,82.0,10.0,22.0,22.0,37.0,14.0,33.0,25.0,13.0,5.0,21.0,25.0,Loss,240.0,34.0,80.0,0.0,8.0,18.0,27.0,12.0,36.0,17.0,9.0,4.0,22.0,31.0
1,2000,1999-11-02,1999-11-02 05:00:00+00:00,Dallas Mavericks,Golden State Warriors,108,96,Win,240.0,39.0,85.0,2.0,15.0,28.0,34.0,11.0,30.0,22.0,11.0,6.0,13.0,19.0,Loss,240.0,34.0,82.0,7.0,18.0,21.0,27.0,18.0,33.0,14.0,4.0,2.0,23.0,29.0
2,2000,1999-11-02,1999-11-02 05:00:00+00:00,Denver Nuggets,Phoenix Suns,107,102,Win,265.0,40.0,89.0,8.0,18.0,19.0,26.0,13.0,42.0,25.0,6.0,16.0,18.0,17.0,Loss,265.0,43.0,111.0,9.0,22.0,7.0,14.0,21.0,37.0,29.0,9.0,5.0,13.0,28.0
3,2000,1999-11-02,1999-11-02 05:00:00+00:00,Houston Rockets,Milwaukee Bucks,93,98,Loss,240.0,34.0,75.0,6.0,20.0,19.0,29.0,15.0,27.0,22.0,5.0,7.0,18.0,24.0,Win,240.0,38.0,79.0,3.0,7.0,19.0,28.0,10.0,25.0,20.0,10.0,2.0,8.0,25.0
4,2000,1999-11-02,1999-11-02 05:00:00+00:00,Los Angeles Clippers,Seattle Supersonics,92,104,Loss,240.0,35.0,78.0,6.0,16.0,16.0,27.0,12.0,33.0,14.0,4.0,9.0,17.0,24.0,Win,240.0,37.0,93.0,7.0,20.0,23.0,32.0,19.0,34.0,25.0,8.0,3.0,11.0,21.0


<a name="2.-Create-Team-Targets"></a>
# 2. Create Team Targets

[Return to top](#Feature-Engineering)

<a name="2.1.-Game-Winner"></a>
## 2.1. Game Winner

[Return to top](#Feature-Engineering)

This can either be `team_bs_df['outcome']`, or it can be `team_matchups_df['home_outcome']` and `team_matchups_df['away_outcome']`.

In [43]:
team_bs_df[['date', 'team', 'outcome']].head()

,date,team,outcome
0,1999-11-02,Orlando Magic,Loss
1,1999-11-02,Charlotte Hornets,Win
2,1999-11-02,Golden State Warriors,Loss
3,1999-11-02,Dallas Mavericks,Win
4,1999-11-02,Phoenix Suns,Loss


In [44]:
team_matchups_df[['date', 'home_team', 'home_outcome', 'away_team', 'away_outcome']].head()

,date,home_team,home_outcome,away_team,away_outcome
0,1999-11-02,Charlotte Hornets,Win,Orlando Magic,Loss
1,1999-11-02,Dallas Mavericks,Win,Golden State Warriors,Loss
2,1999-11-02,Denver Nuggets,Win,Phoenix Suns,Loss
3,1999-11-02,Houston Rockets,Loss,Milwaukee Bucks,Win
4,1999-11-02,Los Angeles Clippers,Loss,Seattle Supersonics,Win


<a name="2.2.-Total-Points-(over-/-under)"></a>
## 2.2. Total Points (over / under)

[Return to top](#Feature-Engineering)

This can be calculated as `team_matchups_df['home_team_score'] + team_matchups_df['away_team_score']`.

In [156]:
# create a new column with the total score
team_matchups_df['score_total'] = team_matchups_df['home_team_score'] + team_matchups_df['away_team_score']

In [157]:
team_matchups_df[['date', 'home_team', 'home_team_score', 'away_team', 'away_team_score', 'score_total']].head()

,date,home_team,home_team_score,away_team,away_team_score,score_total
0,1999-11-02,Charlotte Hornets,100,Orlando Magic,86,186
1,1999-11-02,Dallas Mavericks,108,Golden State Warriors,96,204
2,1999-11-02,Denver Nuggets,107,Phoenix Suns,102,209
3,1999-11-02,Houston Rockets,93,Milwaukee Bucks,98,191
4,1999-11-02,Los Angeles Clippers,92,Seattle Supersonics,104,196


<a name="2.3.-Difference-in-Points-(plus-/-minus)"></a>
## 2.3. Difference in Points (plus / minus)

[Return to top](#Feature-Engineering)

This can be calculated as `team_matchups_df['home_team_score'] - team_matchups_df['away_team_score']`.

In [158]:
# create a new column with the score difference
team_matchups_df['score_diff'] = team_matchups_df['home_team_score'] - team_matchups_df['away_team_score']

In [159]:
team_matchups_df[['date', 'home_team', 'home_team_score', 'away_team', 'away_team_score', 'score_diff']].head()

,date,home_team,home_team_score,away_team,away_team_score,score_diff
0,1999-11-02,Charlotte Hornets,100,Orlando Magic,86,14
1,1999-11-02,Dallas Mavericks,108,Golden State Warriors,96,12
2,1999-11-02,Denver Nuggets,107,Phoenix Suns,102,5
3,1999-11-02,Houston Rockets,93,Milwaukee Bucks,98,-5
4,1999-11-02,Los Angeles Clippers,92,Seattle Supersonics,104,-12


In [160]:
# write out the matchups with merged features
team_matchups_df.to_csv('../data/processed/br_team_matchups_2000_2024.csv', index=False)

<a name="3.-Additional-Team-Level-Features"></a>
# 3. Additional Team Level Features

[Return to top](#Feature-Engineering)

<a name="4.-Player-Level-Features"></a>
# 4. Player Level Features

[Return to top](#Feature-Engineering)

<a name="5.-Time-Windowed-Features"></a>
# 5. Time Windowed Features

[Return to top](#Feature-Engineering)